In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
from pprint import pprint
import requests
import json
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)


#reading raw data
data1 = pd.read_csv("Inpatient_Prospective_Payment_System.csv",encoding='mac_roman')
data2 = pd.read_csv("bedsLatLong.csv",encoding='mac_roman')
data3 = pd.read_csv("ratingsAndOwnership.csv",encoding='mac_roman')


In [152]:
#selecting the columns 
data1 = data1[['Provider Id', 'Provider Name',
       'Provider Street Address', 'Provider City', 'Provider State',
       'Provider Zip Code',     
       ' Average Total Payments ', 'Average Medicare Payments']]


data2=data2[[ 'ID','NAME',  'BEDS']]

data2 = data2.rename(columns={'ID': 'Provider Id', 'NAME':'Provider Name'})

data3= data3[['Provider ID', 'Hospital Type', 'Hospital Ownership',
      
       'Hospital overall rating'
      ]]

data3 = data3.rename(columns={'Provider ID': 'Provider Id'})



In [153]:
#Merg the initial data sets
main = pd.merge(data1,data3, on = 'Provider Id', how = 'outer')
main = pd.merge(main,data2, on = 'Provider Name', how = 'outer')




In [154]:
#do some clean up
main= main.drop_duplicates()

main = main.dropna(how = "any") 

main = main[['Provider Id_x', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       ' Average Total Payments ', 'Average Medicare Payments',
       'Hospital Type', 'Hospital Ownership', 'Hospital overall rating',
        'BEDS']]
main = main.rename(columns={'Provider Id_x': 'Provider Id'})

In [155]:
main.columns

Index(['Provider Id', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       ' Average Total Payments ', 'Average Medicare Payments',
       'Hospital Type', 'Hospital Ownership', 'Hospital overall rating',
       'BEDS'],
      dtype='object')

In [156]:
#grouping catagory into Hospital ownership: Voluntary(non-profit), Government, Individual business
main["Hospital Ownership"] = main["Hospital Ownership"].replace({
    'Voluntary non-profit - Private':'Voluntary(non-profit)',
    'Voluntary non-profit - Church':'Voluntary(non-profit)',
    'Voluntary non-profit - Other':'Voluntary(non-profit)', 
    'Government - Hospital District or Authority': 'Government',
    'Government - State': 'Government',
    'Government - Local': 'Government',
    'Government - Federal': 'Government',
    'Proprietary': 'Individual Business',
     'Physician': 'Individual Business'
                                 
                                 })

In [157]:
#cleaning up data
main = main[main['Hospital overall rating'] != "Not Available"]

In [158]:
# removing the dollars sign and turn these two columns to float datatype
main[' Average Total Payments '] = main[' Average Total Payments '].str.split('$').apply(lambda x: x[-1])

main['Average Medicare Payments'] = main['Average Medicare Payments'].str.split('$').apply(lambda x: x[-1])

main[' Average Total Payments '] = pd.to_numeric(main[' Average Total Payments '])
main['Average Medicare Payments'] = pd.to_numeric(main['Average Medicare Payments'])
main['Hospital overall rating'] = pd.to_numeric(main['Hospital overall rating'])
# df.dtypes

In [159]:
#Adding a new column "cost" this is the amount that patient has to pay
main['Cost'] = main[' Average Total Payments '] - main['Average Medicare Payments']

In [160]:
#clean up
main['Provider Id'] = main['Provider Id'] .astype(np.int64)

In [161]:
#prepping for lat and long 

# create a seies name location 
main['location']= main['Provider Street Address'] + ", " + main['Provider City'] + ", " + main['Provider State']



#### UNCOMMENT AT YOUR OWN RISK , THIS IS FOR GETTING COORDINATES FOR HOSPITAL LOCATIONS USING GOOGLE API

In [162]:
# #only search unique location using google api to prevent rate limiting issue
# unique = main['location'].unique().tolist()


# #store coordinates
# lat = []
# lng = []

# #run google api using the unique locations
# for target_address in unique:
#     params = {"address": target_address, "key": api_key}
#     base_url = "https://maps.googleapis.com/maps/api/geocode/json"
#     response = requests.get(base_url, params=params).json()
#     #traverse response to find lon/lat
#     try:
#         lat.append(response['results'][0]['geometry']['location']['lat'])
#         lng.append(response['results'][0]['geometry']['location']['lng'])
       
#     except Exception:
#         lat.append(0)
#         lng.append(0)
#     print (f"{len(lat)/len(unique)*100}%")  #progress counter

# #add coordinates and unique loation to data frame
# tdf = pd.DataFrame({'Latitude':lat, "Longitude": lng, "location": unique})


# #save dataframe to csv, this way we can comment the google api section out 
# tdf.to_csv("lat_long_location.csv")

In [163]:
#read the data we just made for coordinates
data4 = pd.read_csv("lat_long_location.csv",encoding='mac_roman')
main = pd.merge(main,data4, on = 'location', how = 'outer')

In [164]:
#clean up
main= main.drop_duplicates()
main = main.dropna(how = "any") 
main= main[['Provider Id', 'Provider Name', 'Provider Street Address',
       'Provider City', 'Provider State', 'Provider Zip Code',
       ' Average Total Payments ', 'Average Medicare Payments',
       'Hospital Type', 'Hospital Ownership', 'Hospital overall rating',
       'BEDS', 'Cost', 'location', 'Latitude', 'Longitude']]

main = main[main['Longitude'] != 0]  # getting rid of 0 coordinates 

In [165]:
main.isnull().sum() # no missing values

Provider Id                  0
Provider Name                0
Provider Street Address      0
Provider City                0
Provider State               0
Provider Zip Code            0
 Average Total Payments      0
Average Medicare Payments    0
Hospital Type                0
Hospital Ownership           0
Hospital overall rating      0
BEDS                         0
Cost                         0
location                     0
Latitude                     0
Longitude                    0
dtype: int64

In [166]:
main.shape #clean data

(121431, 16)

In [167]:
main.dtypes #data types are good

Provider Id                    int64
Provider Name                 object
Provider Street Address       object
Provider City                 object
Provider State                object
Provider Zip Code            float64
 Average Total Payments      float64
Average Medicare Payments    float64
Hospital Type                 object
Hospital Ownership            object
Hospital overall rating        int64
BEDS                         float64
Cost                         float64
location                      object
Latitude                     float64
Longitude                    float64
dtype: object

In [168]:
main.count() #all columns have the same length

Provider Id                  121431
Provider Name                121431
Provider Street Address      121431
Provider City                121431
Provider State               121431
Provider Zip Code            121431
 Average Total Payments      121431
Average Medicare Payments    121431
Hospital Type                121431
Hospital Ownership           121431
Hospital overall rating      121431
BEDS                         121431
Cost                         121431
location                     121431
Latitude                     121431
Longitude                    121431
dtype: int64

In [169]:
#export the main data 
main.to_csv("main_data.csv")

In [1]:
main.tail()

NameError: name 'main' is not defined